In [ ]:
!pip install keras-tuner

In [3]:
import numpy as np

import tensorflow as tf
from tensorflow import keras
from keras_tuner.tuners import RandomSearch
from keras_tuner.engine.hyperparameters import HyperParameters

In [4]:
tf.__version__

'2.8.2'

In [5]:
fashion_df = keras.datasets.fashion_mnist

(train_images, train_labels), (test_images, test_labels) = fashion_df.load_data()

4431872/4422102 [==============================] - 0s 0us/step


In [6]:
train_images = train_images/255.0
test_images = test_images/255.0

In [7]:
train_images[0].shape 

(28, 28)

In [8]:
len(train_images)

60000

In [9]:
train_images = train_images.reshape(len(train_images), 28, 28, 1)
test_images = test_images.reshape(len(test_images), 28, 28, 1)

In [10]:
def build_model(hp):
  model = keras.Sequential([
      keras.layers.Conv2D(
          filters=hp.Int("conv_1_filter", min_value=32, max_value=128, step=16), 
          kernel_size=hp.Choice("conv_1_kernel", values=[3, 5]), 
          activation="relu", 
          input_shape=(28, 28, 1)                      
      ), 
      keras.layers.Conv2D(
          filters=hp.Int("conv_2_filter", min_value=32, max_value=64, step=16), 
          kernel_size=hp.Choice("conv_2_kernel", values=[3, 5]), 
          activation="relu", 
      ), 
      keras.layers.Flatten(), 
      keras.layers.Dense(
          units=hp.Int("dense_1_units", min_value=32, max_value=128, step=16), 
          activation="relu"
      ), 
      keras.layers.Dense(units=10, activation="softmax")
  ])

  model.compile(
      optimizer=keras.optimizers.Adam(hp.Choice("learning_rate", values=[1e-2, 1e-3])), 
      loss="sparse_categorical_crossentropy", 
      metrics=["accuracy"]
  )

  return model

In [12]:
tuner = RandomSearch(
    build_model, 
    objective="val_accuracy", 
    max_trials=5, 
    # executions_per_trail=3, 
    directory="CNN Output", 
    project_name="CNN MNIST"
)

In [13]:
tuner.search(train_images, train_labels, 
             epochs=5, 
             validation_split=0.1)

Trial 5 Complete [00h 00m 42s]
val_accuracy: 0.8790000081062317

Best val_accuracy So Far: 0.9128333330154419
Total elapsed time: 00h 04m 12s
INFO:tensorflow:Oracle triggered exit


In [16]:
model = tuner.get_best_models(num_models=1)[0]

In [17]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 24, 24, 32)        832       
                                                                 
 conv2d_1 (Conv2D)           (None, 20, 20, 48)        38448     
                                                                 
 flatten (Flatten)           (None, 19200)             0         
                                                                 
 dense (Dense)               (None, 128)               2457728   
                                                                 
 dense_1 (Dense)             (None, 10)                1290      
                                                                 
Total params: 2,498,298
Trainable params: 2,498,298
Non-trainable params: 0
_________________________________________________________________


In [19]:
model.fit(
    train_images, 
    train_labels, 
    epochs = 25, 
    validation_split=0.2, 
    initial_epoch=5
)

Epoch 6/25
1500/1500 [==============================] - 7s 4ms/step - loss: 0.1189 - accuracy: 0.9552 - val_loss: 0.2030 - val_accuracy: 0.9310
Epoch 7/25
1500/1500 [==============================] - 6s 4ms/step - loss: 0.0891 - accuracy: 0.9663 - val_loss: 0.2243 - val_accuracy: 0.9291
Epoch 8/25
1500/1500 [==============================] - 6s 4ms/step - loss: 0.0660 - accuracy: 0.9756 - val_loss: 0.2659 - val_accuracy: 0.9233
Epoch 9/25
1500/1500 [==============================] - 6s 4ms/step - loss: 0.0478 - accuracy: 0.9830 - val_loss: 0.2821 - val_accuracy: 0.9209
Epoch 10/25
1500/1500 [==============================] - 6s 4ms/step - loss: 0.0387 - accuracy: 0.9864 - val_loss: 0.3441 - val_accuracy: 0.9252
Epoch 11/25
1500/1500 [==============================] - 6s 4ms/step - loss: 0.0365 - accuracy: 0.9867 - val_loss: 0.3540 - val_accuracy: 0.9224
Epoch 12/25
1500/1500 [==============================] - 6s 4ms/step - loss: 0.0280 - accuracy: 0.9901 - val_loss: 0.3953 - val_accura